In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
from urllib.request import urlretrieve
import json
import tabula as tb
from datetime import datetime
import re

#ignore warnings for this notebook
import warnings
warnings.filterwarnings("ignore")

## TSA

In [65]:
#Read directly from the PDF

#Tabula provides ways to filter what part of the page you are looking at using the area parameter. 
# With the area parameter you can specify a rectangular area of the PDF to use. It follows (top, left, bottom, right)


trial_pdf = tb.read_pdf('tsa.pdf', pages='1', area = (320, 510, 1000, 1800), columns = [510, 700, 850, 1000, 1250, 1400, 1650])

In [67]:
tsa_pdf = trial_pdf[0]
tsa_pdf.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [68]:
tsa_pdf

,Date,2023,2022,2021,2020,2019
0,2/10/2023,"2,246,023","1,918,436","1,151,420","2,507,588","1,620,239"
1,2/9/2023,"2,152,183","1,786,323","1,034,514","2,415,185","2,055,827"
2,2/8/2023,"1,793,794","1,390,835","735,009","2,038,375","2,105,696"
3,2/7/2023,"1,622,865","1,239,137","617,619","1,814,047","1,648,328"
4,2/6/2023,"2,033,636","1,592,651","864,783","2,164,951","2,134,744"
5,2/5/2023,"2,195,570","1,786,681","854,636","2,224,826","2,064,640"
6,2/4/2023,"1,669,778","1,399,603","705,951","1,770,241","1,756,152"
7,2/3/2023,"2,102,998","1,583,678","868,624","2,271,551","1,655,868"
8,2/2/2023,"2,035,923","1,342,502","778,065","2,167,857","2,001,482"
9,2/1/2023,"1,604,546","1,196,728","618,615","1,843,877","1,792,677"


## NYC Taxes

In [17]:
#We need to find the right area of the pdf to read from. Formatted like (top,left,bottom,right)
# We can also create as many columns as we'd like. 
df_pdf = tb.read_pdf('StatmentSearch.pdf', pages='2', area = (50, 0, 1000, 1000), columns = [200, 305, 365, 455, 500])
df_pdf = df_pdf[0]

Got stderr: Feb 12, 2023 11:00:32 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider loadDiskCache
Feb 12, 2023 11:00:32 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Feb 12, 2023 11:00:32 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider addTrueTypeFont
java.io.EOFException
	at java.io.RandomAccessFile.readShort(RandomAccessFile.java:717)
	at org.apache.fontbox.ttf.RAFDataStream.readSignedShort(RAFDataStream.java:77)
	at org.apache.fontbox.ttf.TTFDataStream.read32Fixed(TTFDataStream.java:49)
	at org.apache.fontbox.ttf.TTFParser.parse(TTFParser.java:132)
	at org.apache.fontbox.ttf.OTFParser.parse(OTFParser.java:79)
	at org.apache.fontbox.ttf.OTFParser.parse(OTFParser.java:27)
	at org.apache.fontbox.ttf.TTFParser.parse(TTFParser.java:87)
	at org.apache.fontbox.ttf.OTFParser.parse(OTFParser.java:67)
	at org.apache.pdfbox.pdmodel.font.FileSystemFontProvider.addTrueTypeFont(FileSystemFontProvider.java:651)
	at org.apache.pdfbox.pdmodel.font.FileSystemFontProv

In [18]:
df_pdf.head(15)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Page 2
0,Previous Charges,NaN,NaN,NaN,NaN,Amount
1,Total previous charges including in,terest and payments,NaN,NaN,NaN,$0.00
2,Current Charges,Activity Date,Due Date,NaN,NaN,Amount
3,Finance-Property Tax,NaN,01/01/2020,NaN,NaN,"$20,671.52"
4,Adopted Tax Rate,NaN,NaN,NaN,NaN,$-455.64
5,Rent Stabilization * fee $10/apt.,# Apts,NaN,RS fee identifiers,NaN,NaN
6,Rent Stabilization Fee- Chg,117,01/01/2020,42946900,NaN,"$1,170.00"
7,Rent Stabilization Fee- Chg,117,01/01/2020,42946900,NaN,"$1,170.00"
8,Payment,09/18/2019,NaN,NaN,NaN,"$-1,170.00"
9,Total current charges,NaN,NaN,NaN,NaN,"$21,385.88"


In [19]:
#find the row that contains the number of apartments
index_to_find = df_pdf.loc[df_pdf['Unnamed: 1'] == '# Apts'].index
index_to_find

Int64Index([5], dtype='int64')

In [20]:
#Filter full table for above index
df_pdf.columns = df_pdf.iloc[index_to_find[0]]
df_pdf = df_pdf.iloc[index_to_find[0]+1]
df_pdf

5
Rent Stabilization *  fee $10/apt.    Rent Stabilization Fee- Chg
# Apts                                                        117
NaN                                                    01/01/2020
RS fee identifiers                                       42946900
NaN                                                           NaN
NaN                                                     $1,170.00
Name: 6, dtype: object

In [25]:
#we can grab the information we want from here
pd.DataFrame(df_pdf).T

5,Rent Stabilization * fee $10/apt.,# Apts,NaN,RS fee identifiers,NaN,NaN
6,Rent Stabilization Fee- Chg,117,01/01/2020,42946900,NaN,"$1,170.00"
